In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

In [3]:
batch_size = 50
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
# Before you can train your model, 
sequences = []
with open("emi_IgG_seqs.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            sequences.append(seq)
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')

In [7]:
## 
average_hidden_list = []
final_hidden_list = []
hs_list = []
final_cell_list = []


num2 = range(0, 50)
x = 0
y = 50
for i in num2:
    num1 = range(x, y)
    for j in num1:
        avg_hidden, final_hidden, final_cell, hs_out = (b.get_rep_hs(sequences[j]))
        average_hidden_list.append(avg_hidden)
        final_hidden_list.append(final_hidden)
        final_cell_list.append(final_cell)
        hs_list.append(hs_out)
    print('rep')
    x = x + 50
    y = y + 50
    


rep


IndexError: list index out of range

In [8]:
average_hidden_pd = pd.DataFrame(np.row_stack(average_hidden_list))
final_hidden_pd = pd.DataFrame(np.row_stack(final_hidden_list))
hidden_state = pd.DataFrame(np.row_stack(hs_list))
print(average_hidden_pd)

          0         1         2         3         4         5         6   \
0   0.027015  0.164898 -0.095397 -0.958142 -0.010078 -0.236827  0.064693   
1   0.031688  0.170154 -0.091886 -0.956440 -0.007128 -0.224934  0.059931   
2   0.034257  0.165529 -0.103984 -0.954611 -0.012372 -0.248871  0.066804   
3   0.022239  0.162517 -0.091178 -0.960029 -0.009571 -0.214631  0.061275   
4   0.034250  0.167868 -0.096229 -0.956829 -0.008828 -0.261779  0.066648   
5   0.037547  0.171651 -0.098534 -0.958842 -0.011785 -0.251119  0.061967   
6   0.037025  0.166425 -0.097731 -0.961130 -0.012274 -0.252360  0.062111   
7   0.036636  0.173243 -0.102084 -0.957635 -0.012108 -0.254210  0.063059   
8   0.036772  0.170704 -0.099990 -0.958330 -0.012328 -0.251342  0.062292   
9   0.027081  0.162695 -0.101678 -0.960295 -0.013924 -0.242058  0.067489   
10  0.037850  0.173193 -0.095608 -0.957640 -0.008575 -0.242471  0.059206   
11  0.037090  0.169973 -0.103351 -0.957457 -0.013731 -0.247628  0.061990   
12  0.036143

In [9]:
average_hidden_pd.to_csv("emi_IgG_reps.csv")
#final_hidden_pd.to_csv("input_finalhidden_pos_4.csv")

In [8]:
import pickle
save_loc = "C:\\Users\\pkinn\\Documents\\UniRep\\full representations\\emi larger set\\"
data_name = 'ins_seq_2'
file_append = '.pickle'


fn = save_loc + data_name + 'avg_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(average_hidden_list, f)

fn = save_loc + data_name + 'final_hidden' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_hidden_list, f)

fn = save_loc + data_name + 'final_cell' + file_append
with open(fn, 'wb') as f:
    pickle.dump(final_cell_list, f)
   
fn = save_loc + data_name + 'hidden_state' + file_append
with open(fn, 'wb') as f:
    pickle.dump(hs_list, f)
   
fn = save_loc + data_name + 'all_output_hs' + file_append
with open(fn, 'wb') as f:
    pickle.dump([average_hidden_list, final_hidden_list, final_cell_list, hs_list], f)